[Reference](https://towardsdatascience.com/creating-a-ui-with-ipywidgets-and-autocalc-2ef8ea4cc6c2)

In [2]:
!pip install autocalc

In [3]:
import ipywidgets as widgets
from autocalc.autocalc import Var, undefined
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt

In [19]:
# The area (box) for the hyperparameters of the model
model_parameter_box=widgets.Output(layout=widgets.Layout(width='33%', border='1px solid gray'))

# The box for the two tabs
result_box=widgets.Output(layout={'width':'67%'})

# In each tab we have an output box:
confusion_matrix_box=widgets.Output()
prediction_box=widgets.Output()

# Let's create and name our tabs
tab = widgets.Tab()
tab.titles=['Confusion Matrix','Prediction']
tab.children=[confusion_matrix_box, prediction_box]
for i, title in enumerate(['Confusion Matrix','Prediction']):
    tab.set_title(i, title)
    
# Put the tab in the result box
with result_box:
    display(tab)
    

display(widgets.HTML('<br><b><center><font size="12">Iris classification</font></center></b>'))
display(widgets.HBox([model_parameter_box, result_box]))
# Note, that not much can be seen on the screen yet, as we did not fill the
# boxes yet. We will do so later in the notebook

HTML(value='<br><b><center><font size="12">Iris classification</font></center></b>')

In [5]:
p=SVC().get_params()
C=Var('C', initial_value = p['C'], widget = widgets.FloatText(), description='Regularization parameter. The strength of the regularization is inversely proportional to C. Must be strictly positive. The penalty is a squared l2 penalty.')
kernel = Var('kernel', initial_value=p['kernel'], widget=widgets.Dropdown(options=['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']), description='Specifies the kernel type to be used in the algorithm. ')

In [6]:
# +-+-+-+-+-+
# First let's create the widget for the degree parameter
degree_widget = widgets.IntSlider(min=1, max=6)

# Then implement a function which enables/disabled this widget based on the value of the kernel parameter,
def set_degree_ability(kernel):
    disabled = kernel != 'poly' 
    degree_widget.disabled = disabled

# and connect this function to the actual kernel parameter.
# The Var below will not be used for its value. It is used to trigger an action.
Var('degree_v', fun=set_degree_ability, inputs=[kernel])

# Finally create the corresponding Var, using the widget we created above.
degree = Var('degree', initial_value=p['degree'],
             widget=degree_widget,
             description='Degree of the polynomial kernel function (‘poly’). Ignored by all other kernels.')

In [7]:
class CVar(Var):
    """
    Conditional Var
    """ 
    def __init__(self, *args, v1, enabled_values,**kwargs):
        """
        :param v1: The Var object which specified whether this Vars widget is enabled
        :param enabled_values: Our widget is enabled if the value of v1 is in this list
        """
        # We could also use super() here, but I prefer to be explicit:
        Var.__init__(self, *args, **kwargs)
        def enable(input):
            disabled = input not in enabled_values
            self.widget.disabled = disabled
        Var('',fun=enable, inputs=[v1])

In [8]:
degree=CVar('degree', initial_value=p['degree'],
         widget=widgets.IntSlider(min=1, max=6),
         description='Degree of the polynomial kernel function (‘poly’). Ignored by all other kernels.',
         v1=kernel,
         enabled_values=['poly'])

In [9]:
gamma_type = Var('gamma', initial_value=p['gamma'],
                 widget=widgets.Dropdown(options=['scale', 'auto', 'custom:'], layout={'width':'90px'}))
gamma_float = CVar('custom gamma', initial_value=0.1, widget=widgets.FloatText(layout={'width':'60px'}), v1=gamma_type,
               enabled_values=['custom:'])

# We created two separate inputs for gamma, but the SVC class only accepts a single gamma as input. So let's create
# this gamma value, which is used as an input as a function of the above two variables:
def gamma_fun(gamma_type, gamma_float):
    return gamma_float if gamma_type == 'custom:' else gamma_type
    
# It will be the following (gamma) variable which we will be used as the input to the SVC constructor.
gamma = Var('real gamma', fun=gamma_fun, inputs=[gamma_type, gamma_float])

coef0 = CVar('coef0', initial_value=p['coef0'], widget=widgets.FloatText(),
         description='Independent term in kernel function. It is only significant in ‘poly’ and ‘sigmoid’.',
         v1=kernel, enabled_values=['poly', 'sigmoid'])

In [10]:
with model_parameter_box:
    display(C)
    display(kernel)
    display(degree)
    display(coef0)
    display(widgets.HBox([gamma_type.widget_set, gamma_float.widget]))

# Loading the data


In [11]:
data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
variable_list = ['sepal length', 'sepal width', 'petal length','petal width']
data_df = pd.read_csv(data_url, names=variable_list+['class'])
test_df, train_df = train_test_split(data_df, test_size=0.2)

# Creating the outputs


In [12]:
def model_fun(C, kernel, degree, gamma, coef0):
    svc = SVC(C=C, kernel=kernel, degree=degree, coef0=coef0)
    svc.fit(train_df[variable_list],train_df['class'])
    return svc

def confusion_matrix_fun(model):
    predicted_y = model.predict(train_df[variable_list])
    cm = confusion_matrix(train_df['class'], predicted_y)
    disp = ConfusionMatrixDisplay(cm, display_labels=model.classes_)
    return disp

In [13]:
model = Var('Model', fun=model_fun, inputs=[C, kernel, degree, gamma, coef0], lazy=True)
cm = Var('Confusion Matrix', fun=confusion_matrix_fun, inputs=[model])

In [14]:
calibrate_button=widgets.Button(description='Calibrate model')
calibrate_button.on_click(lambda x:model.get())
with model_parameter_box:
    display(calibrate_button)

In [15]:
def cm_display(cm):
    confusion_matrix_box.clear_output()
    with confusion_matrix_box:
        if cm is undefined:
            display(widgets.HBox([widgets.Label('Model not calibrated yet. Click to '),calibrate_button]))
        else:
            cm.plot()
            plt.show()

In [16]:
# Note, how we define the "undefined_input_handling" parameter.

# We assign the value to "_" in order to prevent the display. We will not use the Var we created, only use it as 
# a means to trigger an action based on cm
_ = Var('cm_display', fun=cm_display, inputs=[cm], undefined_input_handling='function')

In [17]:
model_input_variables = [Var(name, widget=widgets.FloatText(), initial_value=train_df[name].iloc[0]) for name in variable_list]
with prediction_box:
    for v in model_input_variables:
        display(v)

In [18]:
def prediction_fun(model, *arg):
    # We create a DataFrame wit a single row, in the same format as we have our training dataset
    input_df = pd.DataFrame(columns=variable_list, data=[arg])
    
    # The output of .predict is an array of size, 1. We want to extract this value from the array, hence [0]
    y = model.predict(input_df)[0]
    return y

# We set read_only=True below. This has the effect that it is disallows to change the value of this Var directly
# but only through its inputs.
Predicted=Var('Predicted',fun=prediction_fun, inputs=[model, *model_input_variables],
              widget=widgets.Text(value='Please calibrate the model'),
             read_only=True)

with prediction_box:
    display(Predicted)